In [54]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None 

### Nous ferons ici une analyse économétrique des données comme vu dans les cours de l'ensae. 

In [55]:
file_path='Romain_Scrapping/offres_emploi_concatene_cleaned.csv'

# Nombre attendu de colonnes
expected_columns = 11

# Fonction personnalisée pour lire le fichier
def filter_csv(file_path, delimiter=','):
    with open(file_path, 'r') as file:
        filtered_rows = []
        for line in file:
            row = line.strip().split(delimiter)
            if len(row) == expected_columns:
                filtered_rows.append(row)
    return pd.DataFrame(filtered_rows)

# Chargement et affichage des données filtrées
data = filter_csv(file_path)

#Nomer les colonnes
data.columns = data.iloc[0]
data = data.drop(0).reset_index(drop=True)

### Regression sur les études et l'experiance

In [56]:
data_exp_etude = data[['salaire_annuel', 'etude_requise' , 'experience_requise']]

data_exp_etude['salaire_annuel'] = pd.to_numeric(data_exp_etude['salaire_annuel'], errors='coerce')
data_exp_etude['etude_requise'] = pd.to_numeric(data_exp_etude['etude_requise'], errors='coerce')
data_exp_etude['experience_requise'] = pd.to_numeric(data_exp_etude['experience_requise'], errors='coerce')

# Supprimer les lignes contenant des valeurs manquantes
data_exp_etude = data_exp_etude.dropna(axis=0, how='any')

In [57]:
X = data_exp_etude[['etude_requise' , 'experience_requise']]
y = data_exp_etude[['salaire_annuel']]

# Initialisation et entraînement du modèle
model = LinearRegression()
model.fit(X, y)

X_with_const = sm.add_constant(X)  # Ajout de l'interception
model_stats = sm.OLS(y, X_with_const)  # Création du modèle
results = model_stats.fit()  # Ajustement du modèle

# Résumé détaillé
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         salaire_annuel   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     3092.
Date:                Sat, 28 Dec 2024   Prob (F-statistic):               0.00
Time:                        19:46:03   Log-Likelihood:            -1.1969e+05
No. Observations:               11371   AIC:                         2.394e+05
Df Residuals:                   11368   BIC:                         2.394e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1.461e+04    262

Les résultats de la régression montrent que le niveau d'études requis et l'expérience exigée ont un impact significatif sur le salaire annuel. Le modèle explique 35,2 % de la variance du salaire, avec des coefficients positifs pour les deux variables : chaque niveau d'études supplémentaire augmente le salaire de 2 644,69 €, et chaque année d'expérience ajoutée augmente le salaire de 160,59 €. Les tests statistiques montrent que ces effets sont statistiquement significatifs. 

### Regression général

In [58]:
data_reg = data[['salaire_annuel', 'etude_requise' , 'experience_requise', 'source','departement','contrat','secteur_activite']]

data_reg['salaire_annuel'] = pd.to_numeric(data_reg['salaire_annuel'], errors='coerce')
data_reg['etude_requise'] = pd.to_numeric(data_reg['etude_requise'], errors='coerce')
data_reg['experience_requise'] = pd.to_numeric(data_reg['experience_requise'], errors='coerce')

data_reg = pd.get_dummies(data_reg, columns=['source','departement','contrat','secteur_activite'], dtype=int)
# Supprimer les lignes contenant des valeurs manquantes
data_reg= data_reg.dropna(axis=0, how='any')

In [59]:
cols = data_reg.columns.tolist()
cols.remove('salaire_annuel')

X = data_reg[cols]
y = data_reg[['salaire_annuel']]

# Initialisation et entraînement du modèle
model = LinearRegression()
model.fit(X, y)

X_with_const = sm.add_constant(X)  # Ajout de l'interception
model_stats = sm.OLS(y, X_with_const)  # Création du modèle
results = model_stats.fit()  # Ajustement du modèle

with open("OLS_results_summary.txt", "w") as file:
    file.write(results.summary().as_text())

# Partie prédictive

In [60]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Création du modèle Lasso
alpha_value = 0.1  # Force de la régularisation
lasso_model = Lasso(alpha=alpha_value)

# Entraînement du modèle
lasso_model.fit(X_train, y_train)

# Prédictions
y_pred = lasso_model.predict(X_test)

# Affichage des coefficients
print("Coefficients du modèle :", lasso_model.coef_)
print("Biais (intercept) :", lasso_model.intercept_)

Coefficients du modèle : [ 2.03204618e+03  1.42168892e+02 -6.58777873e+02 -2.23413370e+03
  5.86055422e+03 -3.81195670e+03 -6.48504260e+02  1.19967901e+03
  5.96189107e+03 -1.76880007e+03 -3.71715875e+02  7.69387789e+02
  2.02123726e+03 -0.00000000e+00  1.04760156e+03  1.06687519e+03
 -7.26058851e+02  1.32831533e+03  0.00000000e+00 -8.68580430e+02
  6.49110648e+03 -1.75418032e+03 -1.73026911e+03  1.80583465e+03
  7.40513513e+02  1.31583880e+03 -5.36795350e+02 -2.31177769e+03
  1.82869889e+02 -1.12204910e+03 -9.95745093e+02  9.37776115e+01
  0.00000000e+00 -1.43310329e+02  1.18743547e+02 -1.60761428e+03
 -1.06313209e+03  6.15037152e+02  4.19960695e+02 -2.43238107e+02
  1.00946514e+02 -1.90166416e+03 -9.05823500e+02 -9.00993425e+03
 -1.73194751e+03 -8.36691456e+00  1.15345480e+03  5.15125452e+02
  1.83639432e+02  3.16479863e+02 -8.29331936e+02  2.43171402e+03
  9.69043136e+02 -4.30268044e+02  5.17499384e+03  1.76385305e+03
  2.61702587e+02  2.36906807e+02 -1.82665124e+03  4.97543427e+02


/home/victorien/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.118e+10, tolerance: 1.083e+08
  model = cd_fast.enet_coordinate_descent(


In [61]:
# Calcul du Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((np.array(y_test) - y_pred) / np.array(y_test)).flatten()) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 32.64937675149142
